In [1]:
import pandas as pd
import geopandas as gpd
import requests, json, geojson
from shapely.geometry import shape
import shapely.wkt
import os
import numpy as np

import sys
sys.path.append('../scripts/')
from read_utils import read_file, create_folder, temp_record_query, temp_record_sdf

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


- postcode(shapefile)

In [2]:
poa_sf = gpd.read_file('../data/tables/external_POA/POA_2021_AUST_GDA2020.shp')
poa_sf['geometry'] = poa_sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
poa_gdf = gpd.GeoDataFrame(poa_sf)

In [3]:
len(poa_gdf)

2644

In [4]:
poa_gdf = poa_gdf.drop(poa_gdf.tail(1).index)

In [5]:
poa_null = poa_gdf[poa_gdf.geometry.isna()]

In [6]:
poa_null

,POA_CODE21,POA_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,SHAPE_Leng,SHAPE_Area,geometry
2641,9494,No usual address (Aust.),AUS,Australia,0.0,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.0,0.0,None
2642,9797,Migratory - Offshore - Shipping (Aust.),AUS,Australia,0.0,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.0,0.0,None


In [7]:
poa_gdf = poa_gdf.replace(to_replace='None', value=np.nan)

In [8]:
poa_gdf.head()

,POA_CODE21,POA_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,SHAPE_Leng,SHAPE_Area,geometry
0,0800,0800,AUS,Australia,3.1731,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.081893,0.000264,"POLYGON ((130.83681 -12.45376, 130.83684 -12.4..."
1,0810,0810,AUS,Australia,24.4283,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.241859,0.002031,"POLYGON ((130.89986 -12.36567, 130.89875 -12.3..."
2,0812,0812,AUS,Australia,35.8899,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.278788,0.002983,"POLYGON ((130.91915 -12.40786, 130.91831 -12.4..."
3,0820,0820,AUS,Australia,39.0642,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.409134,0.003248,"POLYGON ((130.85260 -12.43994, 130.85089 -12.4..."
4,0822,0822,AUS,Australia,150775.8030,http://linked.data.gov.au/dataset/asgsed3/POA/...,90.601831,12.564238,"MULTIPOLYGON (((136.56648 -12.08392, 136.56704..."


In [9]:
poa_geo_info = poa_gdf.drop_duplicates('POA_CODE21')

In [10]:
poa_geo_info['POA_CODE21'] = poa_geo_info['POA_CODE21'].astype('int')

In [11]:
poa_geo_info.head()

,POA_CODE21,POA_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,SHAPE_Leng,SHAPE_Area,geometry
0,800,0800,AUS,Australia,3.1731,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.081893,0.000264,"POLYGON ((130.83681 -12.45376, 130.83684 -12.4..."
1,810,0810,AUS,Australia,24.4283,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.241859,0.002031,"POLYGON ((130.89986 -12.36567, 130.89875 -12.3..."
2,812,0812,AUS,Australia,35.8899,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.278788,0.002983,"POLYGON ((130.91915 -12.40786, 130.91831 -12.4..."
3,820,0820,AUS,Australia,39.0642,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.409134,0.003248,"POLYGON ((130.85260 -12.43994, 130.85089 -12.4..."
4,822,0822,AUS,Australia,150775.8030,http://linked.data.gov.au/dataset/asgsed3/POA/...,90.601831,12.564238,"MULTIPOLYGON (((136.56648 -12.08392, 136.56704..."


In [12]:
poa_geo_info

,POA_CODE21,POA_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,SHAPE_Leng,SHAPE_Area,geometry
0,800,0800,AUS,Australia,3.1731,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.081893,0.000264,"POLYGON ((130.83681 -12.45376, 130.83684 -12.4..."
1,810,0810,AUS,Australia,24.4283,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.241859,0.002031,"POLYGON ((130.89986 -12.36567, 130.89875 -12.3..."
2,812,0812,AUS,Australia,35.8899,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.278788,0.002983,"POLYGON ((130.91915 -12.40786, 130.91831 -12.4..."
3,820,0820,AUS,Australia,39.0642,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.409134,0.003248,"POLYGON ((130.85260 -12.43994, 130.85089 -12.4..."
4,822,0822,AUS,Australia,150775.8030,http://linked.data.gov.au/dataset/asgsed3/POA/...,90.601831,12.564238,"MULTIPOLYGON (((136.56648 -12.08392, 136.56704..."
...,...,...,...,...,...,...,...,...,...
2638,7468,7468,AUS,Australia,309.9494,http://linked.data.gov.au/dataset/asgsed3/POA/...,2.114872,0.033768,"MULTIPOLYGON (((145.39701 -42.13911, 145.39774..."
2639,7469,7469,AUS,Australia,5893.5125,http://linked.data.gov.au/dataset/asgsed3/POA/...,11.341555,0.636169,"MULTIPOLYGON (((144.79215 -41.46478, 144.79212..."
2640,7470,7470,AUS,Australia,108.2441,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.781048,0.011725,"POLYGON ((145.57769 -41.78640, 145.57686 -41.7..."
2641,9494,No usual address (Aust.),AUS,Australia,0.0000,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.000000,0.000000,None


- External Postcode Data

In [13]:
external_postcode = pd.read_csv('../data/tables/external_postcode.csv')

In [14]:
external_postcode.head()

,postcode,place_name,state_name,state_code,latitude,longitude,accuracy
0,200,Australian National University,Australian Capital Territory,ACT,-35.2777,149.1189,1.0
1,221,Barton,Australian Capital Territory,ACT,-35.3049,149.1412,4.0
2,2540,Wreck Bay,Australian Capital Territory,ACT,-35.1627,150.6907,4.0
3,2540,Hmas Creswell,Australian Capital Territory,ACT,-35.0280,150.5501,3.0
4,2540,Jervis Bay,Australian Capital Territory,ACT,-35.1333,150.7000,4.0


In [15]:
postcode_list = list(external_postcode.drop_duplicates(['postcode'])['postcode'])

In [16]:
postcode_lat_mean = external_postcode.groupby(['postcode'])["latitude"].agg(["mean"])
postcode_lat_mean = postcode_lat_mean.reset_index()
postcode_lat_mean.rename(columns = {'mean': 'latitude'}, inplace = True)

In [17]:
postcode_lat_mean

,postcode,latitude
0,200,-35.2777
1,221,-35.3049
2,800,-12.4611
3,801,-12.4611
4,803,-12.4348
...,...,...
3307,9023,-27.4679
3308,9464,-33.8816
3309,9726,-33.8816
3310,9943,-21.1534


In [18]:
postcode_long_mean = external_postcode.groupby(['postcode'])["longitude"].agg(["mean"])
postcode_long_mean = postcode_long_mean.reset_index()
postcode_long_mean.rename(columns = {'mean': 'longitude'}, inplace = True)

In [19]:
postcode_long_mean

,postcode,longitude
0,200,149.1189
1,221,149.1412
2,800,130.8418
3,801,130.8418
4,803,130.7443
...,...,...
3307,9023,153.0281
3308,9464,150.9844
3309,9726,150.9844
3310,9943,149.1655


In [20]:
external_lat_and_long = pd.merge(postcode_lat_mean, postcode_long_mean, on='postcode',how='inner')

In [21]:
external_lat_and_long

,postcode,latitude,longitude
0,200,-35.2777,149.1189
1,221,-35.3049,149.1412
2,800,-12.4611,130.8418
3,801,-12.4611,130.8418
4,803,-12.4348,130.7443
...,...,...,...
3307,9023,-27.4679,153.0281
3308,9464,-33.8816,150.9844
3309,9726,-33.8816,150.9844
3310,9943,-21.1534,149.1655


- Postcode in Consumer Dataset

In [22]:
consumer_df = pd.read_parquet('../data/curated/new_consumer_data.parquet')

In [23]:
consumer_df.loc[:,'postcode'] = consumer_df.loc[:,'postcode'].astype('int')
consumer_df.dtypes

consumer_id    object
name           object
address        object
state          object
postcode        int64
gender         object
user_id         int64
dtype: object

In [24]:
POA_df = poa_geo_info.copy()
POA_df.rename(columns = {'POA_CODE21': 'postcode'}, inplace = True)

In [25]:
POA_df.head()

,postcode,POA_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,SHAPE_Leng,SHAPE_Area,geometry
0,800,0800,AUS,Australia,3.1731,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.081893,0.000264,"POLYGON ((130.83681 -12.45376, 130.83684 -12.4..."
1,810,0810,AUS,Australia,24.4283,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.241859,0.002031,"POLYGON ((130.89986 -12.36567, 130.89875 -12.3..."
2,812,0812,AUS,Australia,35.8899,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.278788,0.002983,"POLYGON ((130.91915 -12.40786, 130.91831 -12.4..."
3,820,0820,AUS,Australia,39.0642,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.409134,0.003248,"POLYGON ((130.85260 -12.43994, 130.85089 -12.4..."
4,822,0822,AUS,Australia,150775.8030,http://linked.data.gov.au/dataset/asgsed3/POA/...,90.601831,12.564238,"MULTIPOLYGON (((136.56648 -12.08392, 136.56704..."


In [26]:
conn_by_postcode = pd.merge(consumer_df,POA_df, on='postcode', how='left')

In [27]:
conn_by_postcode

,consumer_id,name,address,state,postcode,gender,user_id,POA_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,SHAPE_Leng,SHAPE_Area,geometry
0,1195503,Yolanda Williams,413 Haney Gardens Apt. 742,WA,6935,Female,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
1,179208,Mary Smith,3764 Amber Oval,NSW,2782,Female,2,2782,AUS,Australia,21.0358,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.450355,0.002046,"POLYGON ((150.37153 -33.69224, 150.37185 -33.6..."
2,1194530,Jill Jones MD,40693 Henry Greens,NT,862,Female,3,0862,AUS,Australia,117528.1520,http://linked.data.gov.au/dataset/asgsed3/POA/...,26.277085,10.032098,"POLYGON ((137.99526 -17.61136, 137.99537 -17.7..."
3,154128,Lindsay Jimenez,00653 Davenport Crossroad,NSW,2780,Female,4,2780,AUS,Australia,38.4603,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.675708,0.003740,"MULTIPOLYGON (((150.34959 -33.71653, 150.34949..."
4,712975,Rebecca Blanchard,9271 Michael Manors Suite 651,WA,6355,Female,5,6355,AUS,Australia,6019.5820,http://linked.data.gov.au/dataset/asgsed3/POA/...,4.511397,0.581007,"POLYGON ((119.13501 -33.34900, 119.13502 -33.3..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499994,1385608,Jessica Avila,508 Miranda Overpass Apt. 218,QLD,4400,Female,499995,4400,AUS,Australia,45.6179,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.438763,0.004166,"POLYGON ((151.83296 -27.47930, 151.83251 -27.4..."
499995,1466964,Steven Thornton,7913 Schwartz Mission Suite 483,VIC,3097,Undisclosed,499996,3097,AUS,Australia,39.7808,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.549912,0.004064,"POLYGON ((145.27479 -37.65657, 145.27450 -37.6..."
499996,1253484,Christy Smith,5681 Zachary Mountain Apt. 060,NSW,2756,Undisclosed,499997,2756,AUS,Australia,1486.8709,http://linked.data.gov.au/dataset/asgsed3/POA/...,3.645486,0.143944,"POLYGON ((150.85755 -33.29378, 150.87272 -33.2..."
499997,175005,Donna Sutton,54140 Jacob Point,VIC,3989,Female,499998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [28]:
temp = conn_by_postcode[conn_by_postcode.geometry.isna()]
print( f'There are {len(conn_by_postcode)} data, and {len(temp)} of them have no null value.')

There are 499999 data, and 83181 of them have no null value.


In [30]:
null_df = temp[['postcode']].drop_duplicates('postcode').reset_index(drop=True)

In [31]:
column_names=["postcode","latitude",'longitude']
null_df = null_df.reindex(columns = column_names)

In [32]:
null_df

,postcode,latitude,longitude
0,6935,NaN,NaN
1,1109,NaN,NaN
2,6849,NaN,NaN
3,1114,NaN,NaN
4,1825,NaN,NaN
...,...,...,...
522,6946,NaN,NaN
523,1139,NaN,NaN
524,6892,NaN,NaN
525,6847,NaN,NaN


- match postcode into POA area

In [33]:
bound = POA_df['geometry'].bounds
poa_geo = POA_df.join(bound)

In [34]:
poa_geo.head()

,postcode,POA_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,SHAPE_Leng,SHAPE_Area,geometry,minx,miny,maxx,maxy
0,800,0800,AUS,Australia,3.1731,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.081893,0.000264,"POLYGON ((130.83681 -12.45376, 130.83684 -12.4...",130.833318,-12.473697,130.851929,-12.452277
1,810,0810,AUS,Australia,24.4283,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.241859,0.002031,"POLYGON ((130.89986 -12.36567, 130.89875 -12.3...",130.841134,-12.402645,130.906028,-12.330046
2,812,0812,AUS,Australia,35.8899,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.278788,0.002983,"POLYGON ((130.91915 -12.40786, 130.91831 -12.4...",130.871549,-12.412751,130.938572,-12.334656
3,820,0820,AUS,Australia,39.0642,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.409134,0.003248,"POLYGON ((130.85260 -12.43994, 130.85089 -12.4...",130.815126,-12.462090,130.917724,-12.395301
4,822,0822,AUS,Australia,150775.8030,http://linked.data.gov.au/dataset/asgsed3/POA/...,90.601831,12.564238,"MULTIPOLYGON (((136.56648 -12.08392, 136.56704...",129.355624,-14.892084,136.984988,-10.905904


In [35]:
for i in null_df.index:
    for j in external_lat_and_long.index:
        if null_df['postcode'][i] == external_lat_and_long['postcode'][j]:
            null_df['latitude'][i] = external_lat_and_long['latitude'][j]
            null_df['longitude'][i] = external_lat_and_long['longitude'][j]

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [36]:
filled_df = null_df.dropna()
null_df = null_df[null_df.latitude.isna()]

In [37]:
filled_df

,postcode,latitude,longitude
0,6935,-31.9000,115.9667
1,1109,-33.8678,151.2073
2,6849,-31.9333,115.8333
3,1114,-33.8678,151.2073
4,1825,-33.8644,151.0397
...,...,...,...
522,6946,-31.7500,115.8000
523,1139,-33.8678,151.2073
524,6892,-31.9587,115.8711
525,6847,-31.9522,115.8614


The postcode coordinates, other than 9999, were found through a search that these postcode addresses were not geographically specific on aupost and will be dealt with in a subsequent section. Then by searching the website we find that postcode 9999 is a letterbox for letters to Father Christmas, a good wish to send, so it does not make sense for use first, pay later and will also be dealt with in the subsequent section.

In [38]:
null_df

,postcode,latitude,longitude
22,6435,NaN,NaN
191,1441,NaN,NaN
282,2013,NaN,NaN
309,1819,NaN,NaN
337,1476,NaN,NaN
338,3399,NaN,NaN
407,3989,NaN,NaN
492,9999,NaN,NaN
499,8785,NaN,NaN


In [39]:
def match_poa_and_postcode(poa_df, pos_df):
    pos_new_df = pos_df.reindex(columns=[*pos_df.columns.tolist(), 'POA_CODE21'], fill_value=np.nan)
    for i in pos_new_df.index:
        for j in poa_df.index:
            if ((pos_new_df['latitude'][i] >= poa_df['miny'][j])& (pos_new_df['latitude'][i] <= poa_df['maxy'][j])
                &(pos_new_df['longitude'][i] >= poa_df['minx'][j]) & (pos_new_df['longitude'][i] <= poa_df['maxx'][j])):
                pos_new_df.loc[i, 'POA_CODE21'] = poa_df['postcode'][j]
    return pos_new_df

In [40]:
mix_df = match_poa_and_postcode(poa_geo, filled_df)

In [43]:
null_match = mix_df[mix_df.POA_CODE21.isna()]

Since 4029 is contained within 4006, manual matching will match this postcode to the POA range of 4006


In [44]:
null_match

,postcode,latitude,longitude,POA_CODE21
452,4029,-27.1962,153.2931,NaN


In [45]:
mix_df.loc[mix_df['postcode'] == 4029, 'POA_CODE21'] = '4006'

In [46]:
mix_df[mix_df.POA_CODE21.isna()]

,postcode,latitude,longitude,POA_CODE21


In [47]:
mix_df.loc[:, 'POA_CODE21'] = mix_df.loc[:,'POA_CODE21'].astype('int')

In [48]:
mix_df

,postcode,latitude,longitude,POA_CODE21
0,6935,-31.9000,115.9667,6055
1,1109,-33.8678,151.2073,2000
2,6849,-31.9333,115.8333,6014
3,1114,-33.8678,151.2073,2000
4,1825,-33.8644,151.0397,2144
...,...,...,...,...
522,6946,-31.7500,115.8000,6065
523,1139,-33.8678,151.2073,2000
524,6892,-31.9587,115.8711,6004
525,6847,-31.9522,115.8614,6000


In [49]:
customer_df = conn_by_postcode.drop_duplicates('postcode').reset_index(drop=True)

In [50]:
customer_df = customer_df.dropna()

In [51]:
customer_df['POA_CODE21'] = customer_df['postcode']

In [52]:
customer_df.loc[:, 'POA_CODE21'] = customer_df.loc[:,'POA_CODE21'].astype('int')

In [53]:
semi_match_table = customer_df[['postcode', 'POA_CODE21']]

In [54]:
final_match_table = pd.concat([mix_df[['postcode', 'POA_CODE21']], semi_match_table])

In [55]:
final_match_table

,postcode,POA_CODE21
0,6935,6055
1,1109,2000
2,6849,6014
3,1114,2000
4,1825,2144
...,...,...
3162,2307,2307
3163,4151,4151
3164,2126,2126
3165,5420,5420


In [56]:
match_poa_table = pd.merge(final_match_table,poa_geo_info, on='POA_CODE21', how='left')

In [57]:
match_poa_table

,postcode,POA_CODE21,POA_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,SHAPE_Leng,SHAPE_Area,geometry
0,6935,6055,6055,AUS,Australia,58.8100,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.619512,0.005605,"POLYGON ((115.96241 -31.89584, 115.96247 -31.8..."
1,1109,2000,2000,AUS,Australia,4.2871,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.157964,0.000418,"POLYGON ((151.20909 -33.87978, 151.20888 -33.8..."
2,6849,6014,6014,AUS,Australia,9.5822,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.192095,0.000914,"POLYGON ((115.77992 -31.93736, 115.77976 -31.9..."
3,1114,2000,2000,AUS,Australia,4.2871,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.157964,0.000418,"POLYGON ((151.20909 -33.87978, 151.20888 -33.8..."
4,1825,2144,2144,AUS,Australia,8.5637,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.153584,0.000834,"POLYGON ((151.03050 -33.86727, 151.02930 -33.8..."
...,...,...,...,...,...,...,...,...,...,...
3153,2307,2307,2307,AUS,Australia,6.5582,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.126924,0.000632,"POLYGON ((151.69370 -32.88993, 151.69316 -32.8..."
3154,4151,4151,4151,AUS,Australia,5.3272,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.122460,0.000487,"POLYGON ((153.06083 -27.48628, 153.06190 -27.4..."
3155,2126,2126,2126,AUS,Australia,8.2603,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.146167,0.000803,"POLYGON ((151.05881 -33.72735, 151.05908 -33.7..."
3156,5420,5420,5420,AUS,Australia,279.6993,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.769746,0.027052,"POLYGON ((138.75281 -33.21345, 138.75280 -33.2..."


In [58]:
match_geo = gpd.GeoDataFrame(match_poa_table)

In [59]:
match_geo.dtypes

postcode         int64
POA_CODE21       int64
POA_NAME21      object
AUS_CODE21      object
AUS_NAME21      object
AREASQKM21     float64
LOCI_URI21      object
SHAPE_Leng     float64
SHAPE_Area     float64
geometry      geometry
dtype: object

In [60]:
match_geo.to_parquet('../data/curated/geo_sa2_pos.parquet')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  """Entry point for launching an IPython kernel.


In [61]:
path = '../data/curated/geo_sa2_pos'
create_folder(path)
match_geo.to_file(path+'/geo_sa2_pos.shp',
           driver='ESRI Shapefile',
           encoding='utf-8')

|> The folder already exist!
|> Files already exist under this folder:
   ['geo_sa2_pos.prj', 'geo_sa2_pos.shx', 'geo_sa2_pos.cpg', 'geo_sa2_pos.shp', 'geo_sa2_pos.dbf']


- Connect pos_data with poa_data

In [74]:
match_geo.head()

,postcode,POA_CODE21,POA_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,SHAPE_Leng,SHAPE_Area,geometry
0,6935,6055,6055,AUS,Australia,58.8100,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.619512,0.005605,"POLYGON ((115.96241 -31.89584, 115.96247 -31.8..."
1,1109,2000,2000,AUS,Australia,4.2871,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.157964,0.000418,"POLYGON ((151.20909 -33.87978, 151.20888 -33.8..."
2,6849,6014,6014,AUS,Australia,9.5822,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.192095,0.000914,"POLYGON ((115.77992 -31.93736, 115.77976 -31.9..."
3,1114,2000,2000,AUS,Australia,4.2871,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.157964,0.000418,"POLYGON ((151.20909 -33.87978, 151.20888 -33.8..."
4,1825,2144,2144,AUS,Australia,8.5637,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.153584,0.000834,"POLYGON ((151.03050 -33.86727, 151.02930 -33.8..."


In [75]:
pos_to_POA = match_geo[['postcode', 'POA_CODE21', 'geometry']]
pos_to_POA.head()

,postcode,POA_CODE21,geometry
0,6935,6055,"POLYGON ((115.96241 -31.89584, 115.96247 -31.8..."
1,1109,2000,"POLYGON ((151.20909 -33.87978, 151.20888 -33.8..."
2,6849,6014,"POLYGON ((115.77992 -31.93736, 115.77976 -31.9..."
3,1114,2000,"POLYGON ((151.20909 -33.87978, 151.20888 -33.8..."
4,1825,2144,"POLYGON ((151.03050 -33.86727, 151.02930 -33.8..."


In [77]:
poa_gdf = pd.read_parquet('../data/curated/geo_pos_population.parquet')

In [78]:
poa_gdf.head()

,POA_CODE21,2020(no.),2021(no.)
0,800,7708.999833965965,7678.999835068511
1,810,36381.99921240709,36215.99921668285
2,812,19924.00036161399,19888.00036054116
3,820,21186.87673475262,21159.06665843644
4,822,35661.58644259583,35569.3627175903


In [79]:
poa_gdf['POA_CODE21'] = poa_gdf['POA_CODE21'].astype('int')

In [80]:
pos_to_POA.count()

postcode      3158
POA_CODE21    3158
geometry      3158
dtype: int64

In [81]:
pos_to_population = pd.merge(pos_to_POA, poa_gdf, on='POA_CODE21', how='left')

In [86]:
pos_to_population

,postcode,POA_CODE21,geometry,2020(no.),2021(no.)
0,6935,6055,"POLYGON ((115.96241 -31.89584, 115.96247 -31.8...",33667.56962451867,35471.46982550462
1,1109,2000,"POLYGON ((151.20909 -33.87978, 151.20888 -33.8...",31245.01595130939,29155.59319547952
2,6849,6014,"POLYGON ((115.77992 -31.93736, 115.77976 -31.9...",23594.15360007898,23626.47316732434
3,1114,2000,"POLYGON ((151.20909 -33.87978, 151.20888 -33.8...",31245.01595130939,29155.59319547952
4,1825,2144,"POLYGON ((151.03050 -33.86727, 151.02930 -33.8...",42033.87774893655,40303.38192096321
...,...,...,...,...,...
3153,2307,2307,"POLYGON ((151.69370 -32.88993, 151.69316 -32.8...",6501.015501855221,6550.554912219706
3154,4151,4151,"POLYGON ((153.06083 -27.48628, 153.06190 -27.4...",18266.18916872741,18116.22932079525
3155,2126,2126,"POLYGON ((151.05881 -33.72735, 151.05908 -33.7...",19496.60061253601,19159.398477729
3156,5420,5420,"POLYGON ((138.75281 -33.21345, 138.75280 -33.2...",171.5448707179398,171.3798840698726


In [82]:
pos_to_population.head()

,postcode,POA_CODE21,geometry,2020(no.),2021(no.)
0,6935,6055,"POLYGON ((115.96241 -31.89584, 115.96247 -31.8...",33667.56962451867,35471.46982550462
1,1109,2000,"POLYGON ((151.20909 -33.87978, 151.20888 -33.8...",31245.01595130939,29155.59319547952
2,6849,6014,"POLYGON ((115.77992 -31.93736, 115.77976 -31.9...",23594.15360007898,23626.47316732434
3,1114,2000,"POLYGON ((151.20909 -33.87978, 151.20888 -33.8...",31245.01595130939,29155.59319547952
4,1825,2144,"POLYGON ((151.03050 -33.86727, 151.02930 -33.8...",42033.87774893655,40303.38192096321


In [83]:
path = '../data/curated/external_pos_population'
create_folder(path)
match_geo.to_file(path+'/external_pos_population.shp',
           driver='ESRI Shapefile',
           encoding='utf-8')

|> Create Successfully!


In [85]:
pos_to_population.to_parquet('../data/curated/external_pos_population.parquet', index=False)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  """Entry point for launching an IPython kernel.
